In [2]:
import pandas as pd 
from IPython.display import display, HTML

import plotly.graph_objs as go
import plotly.express as px

In [3]:
# Reading in our df 
df = pd.read_pickle("../data/clean/records/sleeps/2021-12-27_2022-08-01.csv")

#### Choose if you want to inclue naps or not 
- True = only naps
- False = sleeps only 
- comment out sleeps + naps 
> Default in VisableWhoop -> naps=False

In [4]:
#df = df[df["is_nap"]==True]
df = df[df["is_nap"]==False]

In [5]:
# Filter dates 

# All data after select date
# df = df.loc[df['during_start'] > '2022-05-01'] # cjx data 

# Between two dates 
df = df.loc[df["during_start"].between("2022-05-06", "2022-05-31")]

In [6]:
# Function in order to convert timestamp to HH:MM format for graph presentation
def f(x):
    ts = x.total_seconds()
    hours, remainder = divmod(ts, 3600)
    minutes, seconds = divmod(remainder, 60)
    return ('{:02d}:{:02d}').format(int(hours), int(minutes)) 

In [7]:
# Creating columns that we are going to need for graphing 
df['light_sleep_dur_frmt'] = pd.to_timedelta(df['light_sleep_duration'], unit='ms').apply(f)
df['slow_wave_sleep_dur_frmt'] = pd.to_timedelta(df.slow_wave_sleep_duration, unit='ms').apply(f)
df['rem_sleep_dur_frmt'] = pd.to_timedelta(df.rem_sleep_duration, unit='ms').apply(f)
df['wake_dur_frmt'] = pd.to_timedelta(df.wake_duration, unit='ms').apply(f)
df['arousal_time_frmt'] = pd.to_timedelta(df.arousal_time, unit='ms').apply(f)


df = df.assign(lsd_hrs = df["light_sleep_duration"]/3600000)
df = df.assign(sws_hrs = df["slow_wave_sleep_duration"]/3600000)
df = df.assign(rem_hrs = df["rem_sleep_duration"]/3600000)
df = df.assign(wake_hrs = df["wake_duration"]/3600000)
df = df.assign(ar_hrs = df["arousal_time"]/3600000)

# Changing col names for presentation 
df = df.rename(columns={'lsd_hrs': 'Light Sleep HRs', 'sws_hrs': 'Slow Wave HRs',
                        'rem_hrs': 'REM Sleep HRs', 'wake_hrs': 'Wake HRs',
                        'ar_hrs': 'Arousal Time HRs'})

In [8]:
# View the entire df head for ref 
display(HTML(df.head().to_html()))

,cycle_id,created_at,updated_at,activity_id,score,quality_duration,latency,debt_pre,debt_post,need_from_strain,sleep_need,habitual_sleep_need,disturbances,time_in_bed,light_sleep_duration,slow_wave_sleep_duration,rem_sleep_duration,cycles_count,wake_duration,arousal_time,no_data_duration,in_sleep_efficiency,credit_from_naps,respiratory_rate,sleep_consistency,algo_version,projected_score,projected_sleep,user_id,timezone_offset,percent_recorded,auto_detected,state,responded,source,is_normal,is_significant,is_nap,during_start,during_end,optimal_sleep_time_start,optimal_sleep_time_end,cycle_start,cycle_end,light_sleep_dur_frmt,slow_wave_sleep_dur_frmt,rem_sleep_dur_frmt,wake_dur_frmt,arousal_time_frmt,Light Sleep HRs,Slow Wave HRs,REM Sleep HRs,Wake HRs,Arousal Time HRs
340,224345865,2022-05-07 07:36:56.381,2022-05-07 07:37:05.931,430711434,85,26529151,0,2222175,2280495,1090772,31090142,27777194,9,28284576,20286664,4016942,2225545,5,1792345,1203552,0,0,0,16.679688,89,5.0.0,85,26529152,3802464,-400,1.0,True,complete,False,auto+user,True,False,False,2022-05-06 23:37:21.419,2022-05-07 07:28:18.117,2022-05-06 23:19:34.102000,2022-05-07 08:34:34.102000,2022-05-07,2022-05-08,05:38,01:06,00:37,00:29,00:20,5.635184,1.115817,0.618207,0.497874,0.334320
341,224788831,2022-05-08 09:30:21.534,2022-05-08 09:30:23.527,431381217,92,28708502,261475,2678489,1578552,708614,31164028,27776924,9,31853930,20990393,3498099,4220010,5,3186176,945925,0,0,0,15.703125,70,5.0.0,92,28708502,3802464,-400,1.0,True,complete,False,auto,True,False,False,2022-05-08 00:25:54.224,2022-05-08 09:16:20.276,2022-05-07 23:13:18.117000,2022-05-08 07:58:18.117000,2022-05-08,2022-05-09,05:49,00:58,01:10,00:53,00:15,5.830665,0.971694,1.172225,0.885049,0.262757
342,225168218,2022-05-09 08:32:22.008,2022-05-09 08:32:23.651,431948916,100,31585519,373947,1428513,0,704851,29910019,27776654,19,34436727,22530419,4079483,4975617,6,2891529,2495440,0,0,0,16.289062,70,5.0.0,100,31585520,3802464,-400,1.0,True,complete,False,auto,True,False,False,2022-05-08 22:47:11.594,2022-05-09 08:23:05.600,2022-05-08 23:46:20.276000,2022-05-09 08:46:20.276000,2022-05-09,2022-05-10,06:15,01:07,01:22,00:48,00:41,6.258450,1.133190,1.382116,0.803203,0.693178
343,225582260,2022-05-10 09:04:52.251,2022-05-10 09:04:53.700,432623840,75,21164523,0,0,4513740,409513,28185897,27776384,13,23855729,13267389,3327697,4569437,3,2722177,1549595,0,0,0,15.996094,62,5.0.0,75,21164524,3802464,-400,1.0,True,complete,False,auto,True,False,False,2022-05-10 02:12:49.511,2022-05-10 08:53:49.050,2022-05-09 23:38:05.600000,2022-05-10 08:38:05.600000,2022-05-10,2022-05-11,03:41,00:55,01:16,00:45,00:25,3.685386,0.924360,1.269288,0.756160,0.430443
344,225961980,2022-05-11 08:52:29.533,2022-05-11 08:52:31.894,433256233,81,26897770,56717,3472014,3062667,1774975,33023105,27776115,19,29926432,17956195,5378052,3563523,2,3068729,2380173,0,0,0,17.167969,79,5.0.0,81,26897770,3802464,-400,1.0,True,complete,False,auto,True,False,False,2022-05-11 00:20:57.556,2022-05-11 08:41:11.438,2022-05-10 23:38:49.050000,2022-05-11 08:53:49.050000,2022-05-11,2022-05-12,04:59,01:29,00:59,00:51,00:39,4.987832,1.493903,0.989868,0.852425,0.661159


### Sleep Stage Duration in Hours
> sleep_stage_durations() in VisbaleWhoop

In [9]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.cycle_start,
    y=df["Light Sleep HRs"],
    stackgroup='one',
    text = df['cycle_start'].astype('string') + " - " +df['light_sleep_dur_frmt'].astype('string'),
    hoverinfo = 'text',
    name = "Light Sleep Duration"))


fig.add_trace(go.Scatter(
    x=df.cycle_start,
    y=df["Slow Wave HRs"],
    stackgroup='one',
    text = df['cycle_start'].astype('string') + " - " +df['slow_wave_sleep_dur_frmt'].astype('string'),
    hoverinfo = 'text',
    name = "Slow Wave Sleep Duration"))

fig.add_trace(go.Scatter(
    x=df.cycle_start,
    y=df["REM Sleep HRs"],
    stackgroup='one',
    text = df['cycle_start'].astype('string') + " - " +df['rem_sleep_dur_frmt'].astype('string'),
    hoverinfo = 'text',
    name = "REM Sleep Duration"))

fig.add_trace(go.Scatter(
    x=df.cycle_start,
    y=df["Wake HRs"],
    stackgroup='one',
    text = df['cycle_start'].astype('string') + " - " +df['wake_dur_frmt'].astype('string'),
    hoverinfo = 'text',
    name = "Wake Duration"))

fig.add_trace(go.Scatter(
    x=df.cycle_start,
    y=df["Arousal Time HRs"],
    stackgroup='one',
    text = df['cycle_start'].astype('string') + " - " +df['arousal_time_frmt'].astype('string'),
    hoverinfo = 'text',
    name = "Arousal Time"))


fig.update_xaxes(title='Date')
fig.update_yaxes(title='Hours')
fig.update_layout(title_text="Sleep Stage Durations", title_x=0.5)
fig.show()

### Box plot of Sleep Stages by Day of the Week 
> sleep_stage_daily() in VisableWhoop

In [10]:
df["cycle_start_dayname"] = df["cycle_start"].dt.day_name()

sleep_stages_df2 = df.melt(id_vars="cycle_start_dayname", value_vars=["Light Sleep HRs", "Slow Wave HRs", "REM Sleep HRs", "Wake HRs", "Arousal Time HRs"])

fig = px.box(sleep_stages_df2, x="cycle_start_dayname", y="value", color="variable",
            labels={
                "value": "Hours",
                "cycle_start_dayname": ""},
            color_discrete_map = {
                'Light Sleep HRs' : 'blue',
                'Slow Wave HRs' : 'orange',
                'REM Sleep HRs' : 'green',
                'Wake HRs' : 'red'})


fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_xaxes(categoryorder='array', categoryarray= ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
fig.update_layout(title_text='Sleep Stage Duration by Day', title_x=0.5, legend_title="Sleep Stage Type")

fig.show()

### Respiratory Rate
- mimics app view
> resp_rate() in VisableWhoop

In [11]:
fig = go.Figure()

fig.add_trace(go.Scatter(
x=df.cycle_start,
y=df["respiratory_rate"],
mode="markers+lines",
text = df['respiratory_rate'].round(2),
hoverinfo = 'text'))


fig.update_layout(title_text='Respiratory Rate', title_x=0.5)
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Breaths per Min')

### Sleep Score by Day
- mimics app view
> sleep_score() in VisableWhoop

**Note**: set naps=False to mimic app view

In [12]:
df2 = df[df["is_nap"]==False]

fig = px.bar(df2, x='cycle_start', y='score',
            labels={'cycle_start': 'Date', 'score':'Sleep Score'})

fig.update_traces(text = df2.score, textposition = "outside")
fig.update_layout(title_text='Sleep Score by Day', title_x=0.5)

fig.show()

### Pre Sleep Debt and Post Sleep Debt
> sleep_need_debt() in VisableWhoop

In [13]:
df = df.assign(debtpre_hrs = df["debt_pre"]/3600000)
df = df.assign(debtpost_hrs = df["debt_post"]/3600000)

fig = px.bar(df, x='cycle_start', y=['debtpre_hrs', 'debtpost_hrs'], orientation = "v",  barmode = 'group',)

newnames = {'debtpre_hrs':'Pre-Sleep Debt', 'debtpost_hrs': 'Post-Sleep Debt'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))
fig.update_layout(title_text='Pre & Post Sleep Debt', title_x=0.5, xaxis_title="Date", yaxis_title="Hours", legend_title="Debt Stage")
fig.show()

### Total time in bed vs Quality Duration 
> timeinbed_qualitydur() in VisableWhoop

In [14]:
df['time_in_bed_frmt'] = pd.to_timedelta(df.time_in_bed, unit='ms').apply(f)
df['quality_duration_frmt'] = pd.to_timedelta(df.quality_duration, unit='ms').apply(f)


df = df.assign(timeinbed_hrs = df["time_in_bed"]/3600000)
df = df.assign(qaulitydur_hrs = df["quality_duration"]/3600000)


fig = px.bar(df, x='cycle_start', y=['timeinbed_hrs', 'qaulitydur_hrs'])

newnames = {'timeinbed_hrs':'Time in Bed', 'qaulitydur_hrs': 'Quality Duration'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))


fig.update_layout(barmode='overlay')
fig.update_layout(title_text='Total Time in Bed vs Quality Duration', title_x=0.5, xaxis_title="Date", yaxis_title="Hours")

fig.show()

### Sleep Stages in Histogram view
> sleep_stages_hist() in VisableWhoop

In [15]:
sleep_stages_df = df.melt(id_vars="cycle_start", value_vars=["Light Sleep HRs", "Slow Wave HRs", "REM Sleep HRs", "Wake HRs", "Arousal Time HRs"])


fig = px.histogram(sleep_stages_df, x="value", color="variable")

fig.update_traces(overwrite=True, marker={"opacity": 0.4}) 
fig.update_layout(barmode='overlay', title_text='Sleep Stages', title_x=0.5, legend_title="Sleep Stage Type",
                xaxis_title="Hours", yaxis_title="Count")

fig.show()

### Sleep Need vs Actual Sleep
> sleep_need_actual() in VisableWhoop

**Note**: set naps=False to mimic app view

In [16]:
# create a column with timedelta as total hours, as a float type
df['total_sleep_sec'] = (df.during_end - df.during_start) / pd.Timedelta(seconds=1)
df['total_sleep_sec'] = df['total_sleep_sec'].astype('float64') 
df['total_sleep_hrs'] = df.total_sleep_sec / 3600


df['total_sleep_need_hrs'] = df.sleep_need / 3600000
df['total_sleep_need_hrs_frmt'] = pd.to_timedelta(df.sleep_need, unit='ms').apply(f)


df = df[df["is_nap"]==False]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.cycle_start,
    y=df["qaulitydur_hrs"],
    mode="markers+lines",
    name = "Total Sleep",
    text = df['cycle_start'].astype('string') + " - " + df['quality_duration_frmt'].astype('string'),
    hoverinfo = 'text'))


fig.add_trace(go.Scatter(
    x=df.cycle_start,
    y=df["total_sleep_need_hrs"],
    mode="markers+lines",
    name = "Sleep Need",
    line=dict(color="#90EE90"),
    text = df['cycle_start'].astype('string') + " - " + df['total_sleep_need_hrs_frmt'].astype('string'),
    hoverinfo = 'text'))

fig.update_layout(title_text='Sleep Need vs Actual Sleep', title_x=0.5)
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Hours')

fig.show()